# JWST SI Search for Exoplanet Spectra
## Introduction

This tutorial will illustrate how to use the MAST API to search for JWST science data by values of [FITS](https://fits.gsfc.nasa.gov/fits_standard.html) header keywords, and then retrieve all products for the corresponding observations. 
Searching by SI Keyword values and accessing all data products is not supported in the [MAST Portal](https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html), nor with the [astroquery.mast](https://astroquery.readthedocs.io/en/latest/mast/mast.html) `Observations` class by itself. Rather, we will be using `astroquery.mast`'s `Mast` class to make direct calls to the MAST API. 

Specifically, this tutorial will show you how to:
* Use the `Mast` class of [astroquery.mast](https://astroquery.readthedocs.io/en/latest/mast/mast.html) to search for JWST science files by values of [FITS](https://fits.gsfc.nasa.gov/fits_standard.html) header keywords
* Construct a unique set of Observation IDs to perform a search with the astroquery.mast `Observation` class
* Fetch the unique data products associated with the Observations
* Filter the results for science products
* Download a bash script to retrieve the filtered products

<div class="alert alert-block alert-info">

<span style="color:black">
Here are key distinctions between the two search methods with <a href="https://astroquery.readthedocs.io/en/latest/mast/mast.html">astroquery.mast</a>:
    <ul>
        <li> <b>Advanced Search for Observations:</b> Uses the <code>Observations</code> class to search for data products that match certain metadata values. The <a href="https://mast.stsci.edu/api/v0/_productsfields.html">available metadata</a> upon which to conduct such a search is limited to coordinates, timestamps, and a modest set of instrument configuration information. Returns MAST <code>Observations</code> objects, which are collections of all levels of products (all formats) and all ancillary data products. </li>
        <li> <b>SI Keyword Search:</b> Uses the <code>Mast</code> class to search for FITS products that match values of user-specified keywords, where the set of possible keywords is very large. Returns only FITS products, and only finds highest level of calibrated products (generally, L-2b and L-3). </li>
    </ul>
</span>
</div>

Connecting files that match keyword values to observations is not difficult, but it is a little convoluted. First, you'll use the API to perform a Science Instrument (SI) Keyword Search to find matching product files. The names of these files contain the MAST Observation ID as a sub-string. Then we can use the IDs to perform an advanced `Observation` search for matching Observations.

Here are the steps in the process:
<ul>
    <li><a href="#Imports">Imports</a></li><br>
    Part I
    <li><a href="#Example">Keyword Search for Exoplanet Spectra</a></li>
    <ul>
        <li><a href="#Criteria">Specify Search Criteria</a></li>
        <li><a href="#Timestamp">Timestamp</a></li>
        <li><a href="#KW Search">Execute the Keyword Search</a></li>
    </ul><br>
    Part II
    <li><a href="#Obs IDs">The Observation Search</a></li>
    <ul>
        <li><a href="#Obs Query">Execute the Observation Search</a></li>
    </ul><br>
    Part III
    <li><a href="#Data Products">Query for Data Products</a></li>
    <ul>
        <li><a href="#Product Filters">Filter the Data Products</a></li>
        <li><a href="#Download Products">Download the Data Products</a></li>
            <ul>
                <li><a href="#Login">MAST Login</a></li>
                <li><a href="#Retrieve Files">Retrieve Files</a></li>
            </ul>
    </ul><br>
    <li><a href="#Resources">Additional Resources</a></li>
</ul>


## Imports
<a id="Imports"></a>

The following packages are needed for this tutorial: 
* [astropy.io](https://docs.astropy.org/en/stable/io/fits/index.html?highlight=astropy.io) allows us to open the `.fits` files that we download
* [astropy.table](https://docs.astropy.org/en/stable/table/index.html?highlight=astropy.table) holds the results of our product query and finds the unique files
* [astropy.time](https://docs.astropy.org/en/stable/time/index.html) creates `Time` objects and converts between time representations
* [astroquery.mast](https://astroquery.readthedocs.io/en/latest/mast/mast.html) constructs the queries, retrieves tables of results, and retrieves data products
* [matplotlib.pyplot](https://matplotlib.org/stable/tutorials/index.html) is a common plotting tool that we'll use to look at our results

In [57]:
from astropy.io import fits
from astropy.table import unique, vstack
from astropy.time import Time
from astroquery.mast import Mast,Observations

import matplotlib.pyplot as plt

# I : Keyword Search for Exoplanet Spectra
<a id="Example"></a>

This example shows how to search for [NIRISS spectral time-series observations (TSO)](https://jwst-docs.stsci.edu/jwst-near-infrared-imager-and-slitless-spectrograph/niriss-observing-modes/niriss-single-object-slitless-spectroscopy) taken of transiting exo-planets. The data are from Commissioning or Early Release Science programs, and are therefore public. 

## Specify Search Criteria
<a id="Criteria"></a>

The criteria for SI Keyword searches consists of FITS header keyword name/value pairs. **Learn more about SI keywords from the [JWST Keyword Dictionary](https://mast.stsci.edu/portal/Mashup/Clients/jwkeywords/index.html), and about the supported set of [keyword values](https://mast.stsci.edu/api/v0/_jwst_inst_keywd.html) that can be queried.** With this kind of query it is necessary to construct a specific structure to hold the query parameters. 

The following helper routines translate a simple dictionary (one that is easy to customize in Python) to the required [JSON](https://www.w3schools.com/js/js_json_intro.asp)-style syntax, while the second creates a Min:Max pair of parameters for date-time stamps which, as with all parameters that vary continuously, must be expressed as a range of values in a dictionary. 

In [59]:
def set_params(parameters):
    return [{"paramName" : p, "values" : v} for p, v in parameters.items()]

def set_mjd_range(min, max):
    '''Set time range in MJD given limits expressed as ISO-8601 dates'''
    return {
        "min": Time(min, format='isot').mjd, 
        "max": Time(max, format='isot').mjd
        }

### Timestamp
<a id="Timestamp"></a>

A date range is specified here (though is not strictly needed) to illustrate how to express these common parameters. For historical reasons, the `astroquery.mast` parameter names for timestamps come in pairs: one with a similar name to the corresponding FITS keyword (e.g. `data_obs`), and another with the string <code>_mjd</code> appended (e.g. `date_obs_mjd`). The values are equivalent, but are expressed in ISO-8601 and MJD representations, respectively. 

Change or add keywords and values to the <code>keywords</code> dictionary below to customize your criteria. Note that multiple, discreet-valued parameters are given in a list. As a reminder, if you are unsure of your keyword and keyword value options, see the [Field Descriptions of JWST Instrument Keywords](https://mast.stsci.edu/api/v0/_jwst_inst_keywd.html) and [JWST Keyword Dictionary](https://mast.stsci.edu/portal/Mashup/Clients/jwkeywords/index.html).

In [61]:
# Looking for NIRISS SOSS commissioning and ERA data taken between June 1st and August 4th
keywords = {'category': ['COM','ERS'],
            'exp_type': ['NIS_SOSS'],
            'tsovisit': ['T'],
            'date_obs_mjd': [set_mjd_range('2022-06-01','2022-08-04')]
           }

# Restructuring the keywords dictionary to the MAST syntax
params = {'columns': '*',
          'filters': set_params(keywords)
         }

The following cell displays the constructed parameter object to illustrate the syntax for the query, which is described formally [here](https://mast.stsci.edu/api/v0/_services.html#MastScienceInstrumentKeywordsNircam). 

In [62]:
params

{'columns': '*',
 'filters': [{'paramName': 'category', 'values': ['COM', 'ERS']},
  {'paramName': 'exp_type', 'values': ['NIS_SOSS']},
  {'paramName': 'tsovisit', 'values': ['T']},
  {'paramName': 'date_obs_mjd', 'values': [{'min': 59731.0, 'max': 59795.0}]}]}

<a id="KW Search"></a>
## Execute the SI Keyword Search
This type of query is a little more primitive in [astroquery.mast](https://astroquery.readthedocs.io/en/latest/mast/mast.html) than that for the `Observations` class. Begin by specifying the webservice for the query, which for this case is the SI keyword search for NIRISS called [Mast.Jwst.Filtered.Niriss](https://mast.stsci.edu/api/v0/_services.html#MastScienceInstrumentKeywordsNiriss). Then execute the query with arguments for the service and the search parameters that we created above.

In [78]:
# Calling the SI keyword search service for NIRISS with our parameters
service = 'Mast.Jwst.Filtered.Niriss'
t = Mast.service_request(service, params)

# Let's display the notebook
display_columns = [x for x in t.colnames if x!="s_region"]
t[display_columns].show_in_notebook(display_length=5)

idx,ArchiveFileID,filename,fileSetName,productLevel,act_id,apername,asnpool,asntable,bartdelt,bendtime,bkgdtarg,bkglevel,bkgsub,bmidtime,bstrtime,category,cont_id,datamode,dataprob,date,date_mjd,date_end,date_end_mjd,date_obs,date_obs_mjd,detector,drpfrms1,drpfrms3,duration,effexptm,effinttm,eng_qual,exp_type,expcount,expend,expmid,exposure,expripar,expstart,fastaxis,filter,frmdivsr,gainfact,gdstarid,groupgap,gs_dec,gs_mag,gs_order,gs_ra,gsendtim,gsendtim_mjd,gsstrttm,gsstrttm_mjd,gs_udec,gs_umag,gs_ura,helidelt,hendtime,hga_move,hmidtime,hstrtime,instrume,intarget,is_psf,lamp,mu_dec,mu_epoch,mu_epoch_mjd,mu_ra,nexposur,nextend,nframes,ngroups,nints,nresets,nrststrt,nsamples,numdthpt,nwfsest,obs_id,observtn,obslabel,origin,pcs_mode,pi_name,pps_aper,prd_ver,program,prop_dec,prop_ra,pwfseet,readpatt,sca_num,scicat,sdp_ver,selfref,seq_id,slowaxis,subarray,subcat,subsize1,subsize2,substrt1,substrt2,targ_dec,targ_ra,targname,targoopp,targprop,targtype,targudec,targura,telescop,template,tframe,tgroup,timesys,title,tsample,tsovisit,visit,visit_id,visitend,visitend_mjd,visitgrp,visitsta,visitype,vststart,vststart_mjd,xoffset,yoffset,zerofram,errtype,rois,roiw,wpower,wtype,datamodl,exp_only,exsegnum,exsegtot,intstart,intend,date_beg,date_beg_mjd,obsfoldr,sctarate,opmode,osf_file,expsteng,expsteng_mjd,masterbg,scatfile,srctyapt,tcatfile,texptime,patt_num,pattsize,patttype,pridtpts,subpxpts,crowdfld,engqlptg,oss_ver,noutputs,gs_v3_pa,dirimage,pixfrac,pxsclrt,segmfile,va_dec,va_ra,compress,bkgmeth,cal_ver,cal_vcs,crds_ctx,crds_ver,focuspos,fwcpos,pupil,pwcpos,nrimdtpt,fileSize,checksum,ingestStartDate,ingestStartDate_mjd,ingestCompletionDate,ingestCompletionDate_mjd,FileTypeID,publicReleaseDate,publicReleaseDate_mjd,isRestricted,isItar,isStale,FileSetId,dataURI
0,5359859,jw01091002001_03102_00001-seg001_nis_x1dints.fits,jw01091002001_03102_00001,2b,02,NIS_SUBSTRIP256,jw01091_20220605t072046_pool,jw01091-o002_20220605t072046_tso-spec2_001_asn.json,-4.644577251747251,59735.51024137305,f,nan,--,59735.50515416697,59735.50006696092,COM,--,91,f,/Date(1654449305470)/,59735.71881331019,/Date(1654431289470)/,59735.51029479167,/Date(1654430410430)/,59735.500120717596,NIS,0,0,879.859,659.28,16.482,OK,NIS_SOSS,6,59735.51029479167,59735.50520775463,1,PRIME,59735.5001207176,-2,F277W,1,nan,N4FF005860,0,60.45670326480025,16.48115348815918,1,260.9997561069748,/Date(1654410251219)/,59735.26679652778,/Date(1654410163155)/,59735.26577726852,0.001019614340759045,0.101490780711174,0.000512506902032688,-3.128389036282897,59735.51025892158,f,59735.50517171548,59735.5000845094,NIRISS,f,--,NONE,0.003759,/Date(946684800000)/,51544.0,0.004891999920901248,6,--,1,3,40,1,1,1,--,0.0,V01091002001P0000000003102,2,BD+601753 Time Series,STSCI,FINEGUIDE,"Martel, Andre",NIS_SUBSTRIP256,PRDOPSSOC-051,1091,60.43078055555555,261.2178566666666,59733.83290263889,NISRAPID,496,--,2022_1c,--,1,-1,SUBSTRIP256,NIRISS,2048,256,1,1793,60.43078055555555,261.2178566666666,BD+60 1753,f,BD+60-1753,FIXED,0.1,0.1,JWST,NIRISS Single-Object Slitless Spectroscopy,5.494,5.494,UTC,NIRISS GR700XD Flux Calibration,10.0,t,1,01091002001,/Date(1654431337406)/,59735.51084960648,03,SUCCESSFUL,PRIME_TARGETED_FIXED,/Date(1654410060755)/,59735.26459208333,nan,nan,f,--,nan,nan,nan,--,MultiSpecModel,f,1,1,1,40,/Date(1654430410430)/,59735.500120717596,--,0.000726542821696803,--,2022156T164116307_001_osf.xml,--,nan,--,--,POINT,--,nan,1,--,--,--,--,f,CALCULATED_TRACK_TR_202111,008.004.007.000,1,189.2489595094376,--,nan,nan,--,60.42979823386356,261.2290731968287,--,--,1.4.6,RELEASE,jwst_0860.pmap,11.6.0,0.3310841320000008,314.8602289999994,GR700XD,245.7982180000013,--,25839360,eb4354f091d9f86aca7a1e96c5ab9eb1,/Date(1654449367394)/,59735.71953001157,/Date(1654449375780)/,59735.71962708333,77,/Date(1657830600000)/,59774.854166666664,False,False,True,782651,mast:JWST/product/jw01091002001_03102_00001-seg001_nis_x1dints.fits
1,5361260,jw01091002001_03101_00001-seg002_nis_calints.fits,jw01091002001_03101_00001,2b,01,NI

<a id="Obs IDs"></a>
# II: Construct the Observation Search

The keyword search returns an astropy table of *files* that match the query criteria. We need to construct MAST Observation IDs from the file names in order to query for all JWST *Observations* that match our criteria. This can be derived from the filenames by removing all characters after and including the final underscore character. Here we make a list of unique Observation IDs for the subsequent query. Note that we limit the list to *unique* IDs, as many filenames have common roots.

In [64]:
# Unique file names:
fn = list(set(t['filename']))

# Set of derived Observation IDs:
ids = list(set(['_'.join(x.split('_')[:-1]) for x in fn]))

Print the list of unique ids if you like.

In [65]:
ids

['jw01366-o001_t001_niriss_clear-gr700xd-substrip256',
 'jw01091002001_03102_00001-seg001_nis',
 'jw01091002001_03101_00001-seg001_nis',
 'jw01541-o001_t002_niriss_f277w-gr700xd-substrip256',
 'jw02734002001_04101_00001-seg001_nis',
 'jw02734002001_04101_00001-seg002_nis',
 'jw01091002001_03101_00001-seg002_nis',
 'jw02734001001_04101_00001-seg003_nis',
 'jw01091002001_03101_00001-seg004_nis',
 'jw02734001001_04101_00001-seg004_nis',
 'jw01366-o001_t001_niriss_f277w-gr700xd-substrip256',
 'jw01091-o002_t001_niriss_f277w-gr700xd-substrip256',
 'jw02734002001_04101_00001-seg003_nis',
 'jw02734001001_04101_00001-seg002_nis',
 'jw02734001001_04102_00001-seg001_nis',
 'jw01366001001_04101_00001-seg002_nis',
 'jw02734-o001_t001_niriss_f277w-gr700xd-substrip256',
 'jw01091002001_03101_00001-seg005_nis',
 'jw01541001001_04101_00001-seg002_nis',
 'jw01541-o001_t002_niriss_clear-gr700xd-substrip256',
 'jw01366001001_04102_00001-seg001_nis',
 'jw01366001001_04101_00001-seg004_nis',
 'jw0109100200

### Execute the Query for Observations
<a id="Obs Query"></a>

Now search for Observations that match the list of Observation IDs constructed above. This search uses the [astroquery.mast](https://astroquery.readthedocs.io/en/latest/mast/mast.html) `Observations` class, where the available search criteria are [described here](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html). 

Note that the Observation ID (`obs_id`) criteria queried for here is not to be confused with the `obsid` that was shown in `Crowded_Fields`: The difference between the two criteria are explained in the CAOM Field Descriptions page linked just above.

In [66]:
# Getting the observations using the `obs_id`s extracted above
matched_obs = Observations.query_criteria(instrument_name='Niriss', 
                                          obs_id=ids
                                         )

# Let's display the notebook 
display_cols = [x for x in matched_obs.columns if x!='s_region']
matched_obs[display_cols].show_in_notebook(display_length=5)

idx,dataproduct_type,calib_level,obs_collection,obs_id,target_name,s_ra,s_dec,t_min,t_max,t_exptime,wavelength_region,filters,em_min,em_max,target_classification,obs_title,t_obs_release,instrument_name,proposal_pi,proposal_id,proposal_type,project,sequence_number,provenance_name,jpegURL,dataURL,dataRights,mtFlag,srcDen,intentType,obsid,objID
0,spectrum,3,JWST,jw02734-o002_t002_niriss_f277w-gr700xd-substrip256,WASP-96,1.046407083333333,-47.36064444444446,59751.15883356481,59751.44992677083,906.735,Infrared,F277W/GR700XD,600.0,5000.0,--,JWST Early Release Observation 8,59773.625,NIRISS,"Pontoppidan, Klaus M.",2734,JWST,JWST,--,CALJWST,--,mast:JWST/product/jw02734-o002_t002_niriss_f277w-gr700xd-substrip256_x1dints.fits,PUBLIC,False,nan,science,87602512,165448166
1,spectrum,3,JWST,jw02734-o001_t001_niriss_clear-gr700xd-substrip256,HAT-P-18,256.3463758333334,33.012325,59743.16453266203,59743.49048288194,25776.465,Infrared,CLEAR/GR700XD,500.0,5000.0,--,JWST Early Release Observation 8,59773.625,NIRISS,"Pontoppidan, Klaus M.",2734,JWST,JWST,--,CALJWST,--,mast:JWST/product/jw02734-o001_t001_niriss_clear-gr700xd-substrip256_x1dints.fits,PUBLIC,False,nan,science,87602513,165448187
2,spectrum,3,JWST,jw02734-o002_t002_niriss_clear-gr700xd-substrip256,WASP-96,1.046407083333333,-47.36064444444446,59751.15883356481,59751.438179375,23080.534,Infrared,CLEAR/GR700XD,500.0,5000.0,--,JWST Early Release Observation 8,59773.625,NIRISS,"Pontoppidan, Klaus M.",2734,JWST,JWST,--,CALJWST,--,mast:JWST/product/jw02734-o002_t002_niriss_clear-gr700xd-substrip256_x1dints.fits,PUBLIC,False,nan,science,87602514,165448204
3,spectrum,3,JWST,jw02734-o001_t001_niriss_f277w-gr700xd-substrip256,HAT-P-18,256.3463758333334,33.012325,59743.16453266203,59743.49877990741,604.565,Infrared,F277W/GR700XD,600.0,5000.0,--,JWST Early Release Observation 8,59773.625,NIRISS,"Pontoppidan, Klaus M.",2734,JWST,JWST,--,CALJWST,--,mast:JWST/product/jw02734-o001_t001_niriss_f277w-gr700xd-substrip256_x1dints.fits,PUBLIC,False,nan,science,87602515,165448207
4,spectrum,3,JWST,jw01091-o002_t001_niriss_f277w-gr700xd-substrip256,BD+60 1753,261.2178566666666,60.43078055555555,59735.26814141204,59735.51029479167,879.859,Infrared,F277W/GR700XD,600.0,5000.0,--,NIRISS GR700XD Flux Calibration,59774.8541666,NIRISS,"Martel, Andre",1091,JWST,JWST,--,CALJWST,--,mast:JWST/product/jw01091-o002_t001_niriss_f277w-gr700xd-substrip256_x1dints.fits,PUBLIC,False,nan,science,84416565,167065229
5,spectrum,3,JWST,jw01541-o001_t002_niriss_f277w-gr700xd-substrip256,HAT-P-14,260.1161716666666,38.24216944444444,59738.26121625,59738.55835982639,384.785,Infrared,F277W/GR700XD,600.0,5000.0,--,NIRISS Sensitivity and Stability for Transiting Exoplanet Observations,59774.8541666,NIRISS,"Espinoza, Nestor",1541,JWST,JWST,--,CALJWST,--,mast:JWST/product/jw01541-o001_t002_niriss_f277w-gr700xd-substrip256_x1dints.fits,PUBLIC,False,nan,science,85715273,167070394
6,spectrum,3,JWST,jw01541-o001_t002_niriss_clear-gr700xd-substrip256,HAT-P-14,260.1161716666666,38.24216944444444,59738.26121625,59738.55248649306,22009.691,Infrared,CLEAR/GR700XD,500.0,5000.0,--,NIRISS Sensitivity and Stability for Transiting Exoplanet Observations,59774.8541666,NIRISS,"Espinoza, Nestor",1541,JWST,JWST,--,CALJWST,--,mast:JWST/product/jw01541-o001_t002_niriss_clear-gr700xd-substrip256_x1dints.fits,PUBLIC,False,nan,science,85883915,167070539
7,spectrum,2,JWST,jw01091002001_03101_00001-seg001_nis,BD+60 1753,261.2178566666666,60.43078055555555,59735.27601813657,59735.4988303588,19268.916,Infrared,CLEAR/GR700XD,500.0,5000.0,--,NIRISS GR700XD Flux Calibration,59774.8541666,NIRISS,"Martel, Andre",1091,JWST,JWST,--,CALJWST,mast:JWST/product/jw01091002001_03101_00001-seg001_nis_calints.jpg,mast:JWST/product/jw01091002001_03101_00001-seg001_nis_x1dints.fits,PUBLIC,False,nan,science,84343025,167076113
8,spectrum,2,JWST,jw01091002001_03101_00001-seg004_nis,BD+60 1753,261.2178566666666,60.43078055555555,59735.27601813657,59735.4988303588,19268.916,Infrared,CLEAR

Verify that your query matched at least one observation, or the remaining steps will fail.

In [67]:
print(f'Found {len(matched_obs)} matching Observations.')

Found 14 matching Observations.


<a id="Data Products"></a>
# III: Query for Data Products

Next we'll fetch the data products that are connected to each Observation. Here we take care to fetch the products from Observations a few at a time (in chunks) to avoid server timeouts. This can happen if there are a large number of files in one or more of the matched Observations. A larger batch size will execute faster, but increases the risk of a server timeout.

The following bit of python magic splits a single long list into a list of smaller lists, each of which has a size no larger than `batch_size`.

In [68]:
batch_size = 4
batches = [matched_obs[i:i+batch_size] for i in range(0, len(matched_obs), batch_size)]

Now fetch the constituent products in a list of tables.

In [69]:
t = [Observations.get_product_list(obs) for obs in batches]

We need to stack the individual tables and extract a unique set of file names. This avoids redundancy because Observations often have many files in common (e.g., guide-star files). 

In [70]:
products = unique(vstack(t), keys='productFilename')
print(f'  Number of unique products: {len(products)}')

  Number of unique products: 870


Display the resulting list of files if you like. 

In [79]:
products.show_in_notebook(display_length=5)

idx,obsID,obs_collection,dataproduct_type,obs_id,description,type,dataURI,productType,productGroupDescription,productSubGroupDescription,productDocumentationURL,project,prvversion,proposal_id,productFilename,size,parent_obsid,dataRights,calib_level
0,84343071,JWST,spectrum,jw01091002001_03102_00001-seg001_nis,source/target (L3) : association generator,S,mast:JWST/product/jw01091-o002_20220605t072046_tso-spec2_001_asn.json,INFO,--,ASN,--,CALJWST,1.5.3,1091,jw01091-o002_20220605t072046_tso-spec2_001_asn.json,1969,84416565,PUBLIC,2
1,84343217,JWST,spectrum,jw01091002001_03101_00001-seg002_nis,source/target (L3) : association generator,S,mast:JWST/product/jw01091-o002_20220605t072046_tso-spec2_003_asn.json,INFO,Minimum Recommended Products,ASN,--,CALJWST,1.4.6,1091,jw01091-o002_20220605t072046_tso-spec2_003_asn.json,1969,84343217,PUBLIC,2
2,84343036,JWST,image,jw01091002001_02101_00002-seg001_nis,source/target (L3) : association generator,S,mast:JWST/product/jw01091-o002_20220703t102255_image2_001_asn.json,INFO,--,ASN,--,CALJWST,1.4.6,1091,jw01091-o002_20220703t102255_image2_001_asn.json,1498,84416565,PUBLIC,2
3,84343098,JWST,image,jw01091002001_02101_00001-seg001_nis,source/target (L3) : association generator,S,mast:JWST/product/jw01091-o002_20220703t102255_image2_002_asn.json,INFO,--,ASN,--,CALJWST,1.4.6,1091,jw01091-o002_20220703t102255_image2_002_asn.json,1498,84416565,PUBLIC,2
4,84343195,JWST,image,jw01091002001_02101_00004-seg001_nis,source/target (L3) : association generator,S,mast:JWST/product/jw01091-o002_20220703t102255_image2_003_asn.json,INFO,--,ASN,--,CALJWST,1.4.6,1091,jw01091-o002_20220703t102255_image2_003_asn.json,1498,84416565,PUBLIC,2
5,84343178,JWST,image,jw01091002001_02101_00003-seg001_nis,source/target (L3) : association generator,S,mast:JWST/product/jw01091-o002_20220703t102255_image2_004_asn.json,INFO,--,ASN,--,CALJWST,1.4.6,1091,jw01091-o002_20220703t102255_image2_004_asn.json,1498,84416565,PUBLIC,2
6,84343085,JWST,spectrum,jw01091002001_03101_00001-seg004_nis,source/target (L3) : association generator,S,mast:JWST/product/jw01091-o002_20220703t102255_tso-spec2_002_asn.json,INFO,Minimum Recommended Products,ASN,--,CALJWST,1.4.6,1091,jw01091-o002_20220703t102255_tso-spec2_002_asn.json,1973,84343085,PUBLIC,2
7,84343207,JWST,spectrum,jw01091002001_03101_00001-seg003_nis,source/target (L3) : association generator,S,mast:JWST/product/jw01091-o002_20220703t102255_tso-spec2_003_asn.json,INFO,Minimum Recommended Products,ASN,--,CALJWST,1.4.6,1091,jw01091-o002_20220703t102255_tso-spec2_003_asn.json,1973,84343207,PUBLIC,2
8,84343025,JWST,spectrum,jw01091002001_03101_00001-seg001_nis,source/target (L3) : association generator,S,mast:JWST/product/jw01091-o002_20220703t102255_tso-spec2_004_asn.json,INFO,Minimum Recommended Products,ASN,--,CALJWST,1.4.6,1091,jw01091-o002_20220703t102255_tso-spec2_004_asn.json,1973,84343025,PUBLIC,2
9,84343158,JWST,spectrum,jw01091002001_03101_00001-seg005_nis,source/target (L3) : association generator,S,mast:JWST/product/jw01091-o002_20220703t102255_tso-spec2_005_asn.json,INFO,Minimum Recommended Products,ASN,--,CALJWST,1.4.6,1091,jw01091-o002_20220703t102255_tso-spec2_005_asn.json,1973,84343158,PUBLIC,2


### Filter the Data Products
<a id="Product Filters"></a>

If there are a subset of products of interest (or, a set of products you would like to exclude) there are a number of ways to do that. The cell below applies a filter to select only calibration level 2 and 3 spectral products classified as `SCIENCE` plus the `INFO` files that define product associations; it also excludes guide-star products. See the full set of [Products Field Descriptions](https://mast.stsci.edu/api/v0/_productsfields.html) for the all queryable fields.

In [80]:
# Retrieve level 2 and 3 SCIENCE and INFO products of type spectrum.
filtered_products = Observations.filter_products(products,
                                                 productType=['SCIENCE', 'INFO'],
                                                 dataproduct_type='spectrum',
                                                 calib_level=[2, 3]
                                                )

Display selected columns of the filtered products, if you like.

In [72]:
filtered_products['description','dataURI', 'calib_level', 'size'].show_in_notebook(display_length=10)

idx,description,dataURI,calib_level,size
0,source/target (L3) : association generator,mast:JWST/product/jw01091-o002_20220605t072046_tso-spec2_001_asn.json,2,1969
1,source/target (L3) : association generator,mast:JWST/product/jw01091-o002_20220605t072046_tso-spec2_003_asn.json,2,1969
2,source/target (L3) : association generator,mast:JWST/product/jw01091-o002_20220703t102255_tso-spec2_002_asn.json,2,1973
3,source/target (L3) : association generator,mast:JWST/product/jw01091-o002_20220703t102255_tso-spec2_003_asn.json,2,1973
4,source/target (L3) : association generator,mast:JWST/product/jw01091-o002_20220703t102255_tso-spec2_004_asn.json,2,1973
5,source/target (L3) : association generator,mast:JWST/product/jw01091-o002_20220703t102255_tso-spec2_005_asn.json,2,1973
6,source/target (L3) : association generator,mast:JWST/product/jw01091-o002_20220703t102255_tso3_002_asn.json,3,3439
7,exposure/target (L2b/L3): 1D extracted spectrum per integration,mast:JWST/product/jw01091-o002_t001_niriss_f277w-gr700xd-substrip256_x1dints.fits,3,37440
8,exposure (L2b): 3D calibrated exposure,mast:JWST/product/jw01091002001_03101_00001-seg001_nis_calints.fits,2,2424458880
9,exposure (L2a): 2D count rate averaged over integrations,mast:JWST/product/jw01091002001_03101_00001-seg001_nis_rate.fits,2,10546560


<a id="Download Products"></a>
## Download the Data Products

We'll go over your options for data downloads in the sections below. Note that for public data, you will not need to login.

<a id="Login"></a>
### Optional: MAST Login

If you intend to retrieve data that are protected by an Exclusive Access Period (EAP), you will need to be both *authorized* and *authenticated*. You can authenticate by presenting a valid [Auth.MAST](https://auth.mast.stsci.edu/info) token with the login function. (See [MAST User Accounts](https://outerspace.stsci.edu/display/MASTDOCS/MAST+User+Accounts) for more information about whether you need to login.) Note: this step is unnecessary if you are only retrieving public data. 

<div class="alert alert-block alert-warning">

<span style="color:black">
    If you have arrived at this point, wish to retrieve EAP products, and have <b>not</b> establihed a token, you need to do the following:
    <ul>
        <li> Create a token here: <a href="https://auth.mast.stsci.edu/info">Auth.MAST</a>
        <li> Cut/paste the token string in response to the prompt that will appear when downloading the script. </li>
    </ul>
    Defining the token string as an environment variable <b>will not work</b> for an already-running notebook.
</span>
</div>

In [ ]:
# Observations.login()

<a id="Retrieve Files"></a>
### Retrieve FIles

Now let's fetch the products. The example below shows how to retrieve a bash script (rather than direct file download) to retreive our entire list at once. Scripts are a much better choice if the number of files in the download manifest is large (>100).

In [ ]:
# Downloading via a bash script.

manifest = Observations.download_products(filtered_products,
                                          curl_flag=True
                                         )

In the interest of time (and not crashing our servers), we will download one small product from our list above. Let's download a reasonably sized 10MB file. The file we choose is raw spectral data, so additional extractiong would be needed for scientific analysis.

In [ ]:
manifest = Observations.download_products(filtered_products[9])

We can actually visualize the raw data from which the spectrum can be extracted:

In [ ]:
sci = fits.getdata(manifest['Local Path'][0], 1)

plt.figure(figsize=(15,150))
plt.imshow(sci);

We are, in effect, seeing the [raw spectrum on the detector](https://jwst-docs.stsci.edu/jwst-near-infrared-imager-and-slitless-spectrograph/niriss-observing-modes/niriss-single-object-slitless-spectroscopy#NIRISSSingleObjectSlitlessSpectroscopy-SOSSsubarrays); if you look closely at the section of the brightest line in the upper right corner of the figure, you can see absorbtion lines.

<a id="Resources"></a>
# Additional Resources

The links below take you to documentation that you might find useful when constructing queries.

* [astropy](https://docs.astropy.org/en/stable/index.html) documentation
* [astroquery.mast](https://astroquery.readthedocs.io/en/latest/mast/mast.html) documentation for querying MAST
* [Field Descriptions for JWST Instrument Keywords](https://mast.stsci.edu/api/v0/_jwst_inst_keywd.html)
* [Queryable fields](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html) in the MAST/CAOM database

## About this notebook

This notebook was developed by Archive Sciences Branch staff: chiefly Dick Shaw, with additional editing from Jenny Medina and Thomas Dutkiewicz. For support, please contact the Archive HelpDesk at archive@stsci.edu, or through the [JWST HelpDesk Portal](https://jwsthelp.stsci.edu). 
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>